# Pre procesamiento

## Importar el archivo de tweets en CSV para el entrenamiento
Se importar el archivo y se eliminan las columnas que no se necesitan, se limpian datos innecesarios y con esto se genera un nuevo archivo con los tweets pre procesados

Basado en: https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-11-cnn-word2vec-41f5e28eda74

## Carga de datos
Se cargan los tweets del archivo CSV en un dataframe. 

Se valida que el dataframe no tenga nulls. Si los tiene, se les hace drop.

Además, se cargan listas de stopwords y emoticones para el pre-procesamiento

In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from Preprocesador import Preprocesador

preprocesador = Preprocesador()

#Definir el nombre de las columnas del archivo
cols = ['id','XX','Polaridad','Texto']

#Importar el archivo en memoria 
#Se indica que no tiene encabezados
#Se usan los nombres definidos anteriormente los nombres de las columnas
df = pd.read_csv("tweets.csv",header=None, names=cols)

#Se botan las columnas que no se necesitan
df.drop(['id','XX'],axis=1,inplace=True)

#Se agrega columna de longitud original para validaciones luego del pre proccesamiento
df['LongitudOriginal'] = [len(str(t)) for t in df.Texto]

#Elimina las líneas que no tengan todos los valores definidos
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df.loc[df['Polaridad'] == 'N+', 'Polaridad'] = 'N'
df.loc[df['Polaridad'] == 'P+', 'Polaridad'] = 'P'

#Imprimir las primeras líneas del archivo para validar una carga correcta
df.head()

[':-)', ':-))', '(-:', '((-:', ':)', ':))', '(:', '((:', ':-]', '[-:']
[':-(', ':(', ':-c', ':c', ':-<', ':<', ':-[', ':[', ':-||', '>:[']


,Polaridad,Texto,LongitudOriginal
0,NONE,Salgo de #VeoTV que día más largoooooo,38
1,NEU,@PauladeLasHeras No te libraras de ayudar meno...,63
2,P,@marodriguezb Gracias MAR,25
3,N,Off pensando en el regalito Sinde la que se va...,128
4,P,Conozco a alguien q es adicto al drama! Ja ja ...,65


## Obtener totales por polaridad
Obtenemos una cuenta de cuantos comentarios hay por polaridad

In [2]:
#Imprimir el total de tweets por polaridad
df.Polaridad.value_counts()

P       3044
N       2399
NONE    1543
NEU      738
Name: Polaridad, dtype: int64

## Definición de método que realiza pre-procesamiento
Se define en este método cuáles técnicas de pre-procesamiento se van a realizar

Se imprime en pantalla una muestra del resultado del pre-procesamiento

In [3]:

#Prueba el pre procesamiento en un grupo pequeño de registros
testing = df.Texto[7218:7221]
test_result = []
for t in testing:
    print(t)
    test_result.append(preprocesador.tweetCleaner(text=t))
print(test_result[0])
print(test_result[1])

@noseashetero 1000/10 de verdad a ti que voy a decir petarda que te quiero más que a mí mismo  ✨
@piscolabisaereo @HistoriaNG @SPosteguillo las tengo pero aún no las he leído. Caerán prontito 
Al final han sido 3h  Bueno, mañana tengo fiesta así que.. No me quejo 
noseashetero xnumxxnumx de verdad a ti que voy a decir petarda que te quiero más que a mí mismo emopos
piscolabisaereo historiang sposteguillo las tengo pero aún no las he leído caerán prontito


## Ejecutar el pre-procesamiento
Se ejecuta el pre-procesamiento sobre todos los tweets y se guardan en una lista

In [4]:
print ("Pre procesando tweets...\n")
clean_tweet_texts = []
for i in range(df.Texto.count()):
    if( (i+1)%1000 == 0 ):
        print("%d de %d tweets procesados" % ( i+1, df.Texto.count() ))
    clean_tweet_texts.append(preprocesador.tweetCleaner(str(df['Texto'][i])))
print ("Pre procesamiento completado")

Pre procesando tweets...

1000 de 7724 tweets procesados
2000 de 7724 tweets procesados
3000 de 7724 tweets procesados
4000 de 7724 tweets procesados
5000 de 7724 tweets procesados
6000 de 7724 tweets procesados
7000 de 7724 tweets procesados
Pre procesamiento completado


## Generar dataframe para exportación
Se crea un dataframe con los tweets y se les agrega la polaridad asociada

In [5]:
#Generar dataframe con tweets pre procesados
clean_df = pd.DataFrame(clean_tweet_texts,columns=['Texto'])
clean_df['Polaridad'] = df.Polaridad
clean_df.head()

,Texto,Polaridad
0,salgo de veotv que día más largoo,NONE
1,pauladelasheras no te libraras de ayudar menos...,NEU
2,marodriguezb gracias mar,P
3,off pensando en el regalito sinde la que se va...,N
4,conozco a alguien q es adicto al drama ja ja j...,P


## Validar nulls
Se valida que no queden nulls. En caso que queden, se les hace drop.

In [6]:
#Verificar si luego del pre procesamiento quedaron registros NULL 
#Hay que revisarlos contra los tweets originales y verificar si hay algún error
clean_df[clean_df.isnull().any(axis=1)]

,Texto,Polaridad


In [7]:
#Luego de revisar los registros NULL, se detectó que el único caso era porque no tenía comentario
#Se van a desechar los registros NULL
clean_df.dropna(inplace=True)
clean_df.reset_index(drop=True,inplace=True)
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7724 entries, 0 to 7723
Data columns (total 2 columns):
Texto        7724 non-null object
Polaridad    7724 non-null object
dtypes: object(2)
memory usage: 120.8+ KB


In [8]:
#Guarda los tweets a un archivo con encoding UTF-8
csv = 'tweets_limpios.csv'
clean_df.to_csv(csv,encoding='utf-8')

In [9]:
#Lo carga de nuevo para corroborar que esté bien
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

,Texto,Polaridad
0,salgo de veotv que día más largoo,NONE
1,pauladelasheras no te libraras de ayudar menos...,NEU
2,marodriguezb gracias mar,P
3,off pensando en el regalito sinde la que se va...,N
4,conozco a alguien q es adicto al drama ja ja j...,P


## Generar archivo para embeddings
Se genera un archivo de texto plano para ser leído por el generador de embeddings

### Lo primero es importar el archivo con tweets adicionales

In [10]:
#Definir el nombre de las columnas del archivo
cols = ['id','Usuario','Texto','Fecha','Idioma','?']

#Importar el archivo en memoria 
#Se indica que no tiene encabezados
#Se usan los nombres definidos anteriormente los nombres de las columnas
df = pd.read_csv("general-tweets-unidos.csv",header=None, names=cols)

#Se botan las columnas que no se necesitan
df.drop(['id','Usuario','Fecha','Idioma','?'],axis=1,inplace=True)

#Se agrega columna de longitud original para validaciones luego del pre proccesamiento
df['LongitudOriginal'] = [len(str(t)) for t in df.Texto]

#Elimina las líneas que no tengan todos los valores definidos
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)

#Imprimir las primeras líneas del archivo para validar una carga correcta
print(df.head())
print ("Pre procesando tweets...\n")
clean_tweet_texts = []
for i in range(df.Texto.count()):
    if( (i+1)%1000 == 0 ):
        print("%d de %d tweets procesados" % ( i+1, df.Texto.count() ))
    clean_tweet_texts.append(preprocesador.tweetCleaner(str(df['Texto'][i])))
print ("Pre procesamiento completado")

                                               Texto  LongitudOriginal
0  Portada 'Público', viernes. Fabra al banquillo...               135
1  Grande! RT @veronicacalderon \El periodista es...                84
2  Gonzalo Altozano tras la presentación de su li...               128
3  Mañana en Gaceta: TVE, la que pagamos tú y yo,...               117
4  Qué envidia “@mfcastineiras: Pedro mañana x la...               137
Pre procesando tweets...

1000 de 61798 tweets procesados
2000 de 61798 tweets procesados
3000 de 61798 tweets procesados
4000 de 61798 tweets procesados
5000 de 61798 tweets procesados
6000 de 61798 tweets procesados
7000 de 61798 tweets procesados
8000 de 61798 tweets procesados
9000 de 61798 tweets procesados
10000 de 61798 tweets procesados
11000 de 61798 tweets procesados
12000 de 61798 tweets procesados
13000 de 61798 tweets procesados
14000 de 61798 tweets procesados
15000 de 61798 tweets procesados
16000 de 61798 tweets procesados
17000 de 61798 tweets procesa

In [11]:
#Generar dataframe con tweets pre procesados
clean_df = pd.DataFrame(clean_tweet_texts,columns=['Texto'])
clean_df.head()

,Texto
0,portada público viernes fabra al banquillo por...
1,grande rt veronicacalderon el periodista es al...
2,gonzalo altozano tras la presentación de su li...
3,mañana en gaceta tve la que pagamos tú y yo cu...
4,qué envidia “mfcastineiras pedro mañana x la m...


In [12]:
#Finalmente, guardar un archivo con solo los textos pre-procesados para realizar la vectorización
texto_tweets = []
for i in range(my_df.Texto.count()):
    texto_tweets.append(str(my_df['Texto'][i]))
    
for i in range(clean_df.Texto.count()):
    texto_tweets.append(str(clean_df['Texto'][i]))

#Guarda el archivo
with open('SoloTextos.txt', 'w', encoding='utf-8') as f:
    for line in texto_tweets:
        f.write(line + "\n")
        
print("Archivo con textos guardado")

Archivo con textos guardado
